<a href="https://colab.research.google.com/github/Yetibi/Autoreel/blob/main/CODIGO_UNIFICACION_ARCHIVOS_AUTOREEL_1_Unir_Dia_mes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODIGO CONSOLIDACIÓN ARCHIVOS RAW - AUTOREEL

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ARCHIVO UNIFICADO COMPLETO - Unifica las pestañas diarias a 1 mensual por tipo de movimiento


In [31]:
import pandas as pd

# Configuración
anio = 2024
meses = {"ENERO": 1, "FEBRERO": 2}  # Meses y sus números
nombre_archivo = f"AUTOREEL ENERO {anio}.xlsx"
ruta_base = "/content/drive/MyDrive/AUTOREEL CONSULTORIA/BASES DE DATOS SOURCE/Año 2024 AutoReel"
ruta_archivo = f"{ruta_base}/{nombre_archivo}"
ruta_salida = f"/content/drive/MyDrive/AUTOREEL CONSULTORIA/AUTOREEL_CONSOLIDADO_ENERO_FEBRERO_{anio}.xlsx"

# Días válidos por mes
dias_validos = {
    "ENERO": [f"ENERO {str(i).zfill(2)}" for i in range(9, 32) if i not in [14, 21, 28]],
    "FEBRERO": [f"FEBRERO {str(i).zfill(2)}" for i in range(1, 30) if i not in [4, 11, 18, 25]],
}

# Abrimos el archivo
xl = pd.ExcelFile(ruta_archivo)

# Contenedores por tipo
servicios = []
base_dia = []
gastos = []
adelantos = []
entregas = []
pagos_transferencias_1 = []
pagos_transferencias_2 = []
ingresos_efectivo = []

# Proceso por hoja y mes
for mes, dias in dias_validos.items():
    for hoja in dias:
        if hoja in xl.sheet_names:
            df_raw = xl.parse(hoja, header=None)
            fecha = pd.to_datetime(f"{hoja.split()[1]}/{meses[mes]}/{anio}", dayfirst=True).date()

            # 1. SERVICIO: B2:M39
            tabla_servicio = df_raw.iloc[1:39, 1:13].copy()
            tabla_servicio.columns = df_raw.iloc[1, 1:13]
            tabla_servicio = tabla_servicio.dropna(how="all")
            tabla_servicio["FECHA"] = fecha
            servicios.append(tabla_servicio)

            # 2. BASE DIA: P2
            valor_base = df_raw.iloc[1, 15]
            if pd.notna(valor_base):
                base_dia.append({"FECHA": fecha, "BASE DÍA": valor_base})

            # 3. GASTOS Ò PAGOS PROVEEDORES: O14:P38
            df_gastos = df_raw.iloc[13:38, 14:16].copy()
            df_gastos.columns = df_raw.iloc[13, 14:16]
            df_gastos = df_gastos.dropna(how="all")
            df_gastos["FECHA"] = fecha
            gastos.append(df_gastos)

            # 4. ADELANTOS O PRESTAMOS: R2:S7
            df_adelantos = df_raw.iloc[1:7, 17:19].copy()
            df_adelantos.columns = ["PERSONA", "VALOR"]
            df_adelantos = df_adelantos.dropna(how="all")
            df_adelantos["FECHA"] = fecha
            adelantos.append(df_adelantos)

            # 5. ENTREGA EFECTIVO DIA A: R10:S12
            entrega = df_raw.iloc[9:12, 17:19].copy()
            entrega.columns = ["PERSONA", "VALOR"]
            entrega = entrega.dropna(how="all")
            entrega["FECHA"] = fecha
            entregas.append(entrega)

            # 6. PAGOS TRANSFERENCIAS: R14:T38
            pagos1 = df_raw.iloc[13:38, 17:20].copy()
            pagos1.columns = df_raw.iloc[13, 17:20]
            pagos1 = pagos1.dropna(how="all")
            pagos1["FECHA"] = fecha
            pagos_transferencias_1.append(pagos1)

            # 7. ABONO/BANCO (llamado también transferencias): U2:W9
            #pagos2 = df_raw.iloc[1:9, 20:23].copy()
            #pagos2.columns = ["ORDEN", "EFECTIVO", "TRANSFERENCIA"]
            #pagos2 = pagos2.dropna(how="all")
            #pagos2["FECHA"] = fecha
            #pagos_transferencias_2.append(pagos2)

            # 8. INGRESO EFECTIVO: U10:W12
            ingresos = df_raw.iloc[9:12, 20:23].copy()
            ingresos.columns = ["INGRESO EFECTIVO", "VALOR", "BANCO"]
            ingresos = ingresos.dropna(how="all")
            ingresos["FECHA"] = fecha
            ingresos_efectivo.append(ingresos)

# Convertimos listas a DataFrames
df_servicio = pd.concat(servicios, ignore_index=True)
df_base = pd.DataFrame(base_dia)
df_gastos = pd.concat(gastos, ignore_index=True)
df_adelantos = pd.concat(adelantos, ignore_index=True)
df_entregas = pd.concat(entregas, ignore_index=True)
df_transf1 = pd.concat(pagos_transferencias_1, ignore_index=True)
df_transf2 = pd.concat(pagos_transferencias_2, ignore_index=True)
df_ingresos = pd.concat(ingresos_efectivo, ignore_index=True)

# Guardamos todo
with pd.ExcelWriter(ruta_salida, engine="openpyxl", mode="w") as writer:
    df_servicio.to_excel(writer, sheet_name="SERVICIOS", index=False)
    df_base.to_excel(writer, sheet_name="BASE DÍA", index=False)
    df_gastos.to_excel(writer, sheet_name="GASTOS Ò PAGOS PROVEEDORES", index=False)
    df_adelantos.to_excel(writer, sheet_name="ADELANTOS O PRESTAMOS", index=False)
    df_entregas.to_excel(writer, sheet_name="ENTREGA EFECTIVO DIA A", index=False)
    df_transf1.to_excel(writer, sheet_name="PAGOS TRANSFERENCIAS 1", index=False)
    df_transf2.to_excel(writer, sheet_name="PAGOS TRANSFERENCIAS 2", index=False)
    df_ingresos.to_excel(writer, sheet_name="INGRESO EFECTIVO", index=False)

print("✅ Consolidación completada para enero y febrero.")
print(f"📁 Archivo generado: {ruta_salida}")

ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

2. GENERA PESTAÑA UNIFICADA DE TODAS LAS CATEGORIAS DE MOVIMIENTOS POR MES


In [ ]:
# 🔄 Crear hoja UNIFICADO con categoría
df_servicio["CATEGORIA"] = "SERVICIOS"
df_base["CATEGORIA"] = "BASE DÍA"
df_gastos["CATEGORIA"] = "GASTOS Ò PAGOS PROVEEDORES"
df_adelantos["CATEGORIA"] = "ADELANTOS O PRESTAMOS"
df_entregas["CATEGORIA"] = "ENTREGA EFECTIVO DIA A"
df_transf1["CATEGORIA"] = "PAGOS TRANSFERENCIAS 1"
df_transf2["CATEGORIA"] = "PAGOS TRANSFERENCIAS 2"
df_ingresos["CATEGORIA"] = "INGRESO EFECTIVO"

# Unimos todos los DataFrames con las mismas columnas al final
df_unificado = pd.concat([
    df_servicio,
    df_base,
    df_gastos,
    df_adelantos,
    df_entregas,
    df_transf1,
    df_transf2,
    df_ingresos
], ignore_index=True)

# Reescribimos el archivo agregando la hoja UNIFICADO
with pd.ExcelWriter(ruta_salida, engine="openpyxl", mode="a") as writer:
    df_unificado.to_excel(writer, sheet_name="UNIFICADO", index=False)

print("📌 Hoja 'UNIFICADO' agregada al archivo con formato largo y categoría incluida.")


📌 Hoja 'UNIFICADO' agregada al archivo con formato largo y categoría incluida.
